In [7]:
import numpy as np

np.random.seed(0)


# Example number of hosts
hosts = 3
attrbutes = 5

# Example weights for VM attributes (a1-aN), where N=5 for simplicity

if attrbutes != 5:
    print("Please adjust the attribute_weights for the number of attributes")
    exit(1)
attribute_weights = np.array([0.2, 0.1, 0.3, 0.15, 0.25])


# Example hosts with their current total attributes (randomly generated for 10 hosts, N=5 attributes each)
host_attributes = np.random.rand(hosts, attrbutes) * 200 + 40 # Values between 40-240 for simplicity

print("host_attributes\n", host_attributes)

# Example VM to be scheduled, with its attributes
#vm_attributes = np.array([20, 15, 30, 25, 10])

def generate_vm():
    return np.random.rand(attrbutes) * 20  # Randomly generate VM attributes between 0-20

# Example green energy metrics for hosts (0-1, where 1 is 100% green)
#green_energy = np.array([0.9, 0.7, 0.8])
green_energy = np.random.rand(hosts)
print("random_green_energy\n", green_energy)

# Example current load percentages for hosts
#host_loads = np.array([0.3, 0.7, 0.6])  # 50%, 70%, 60% load
host_loads = np.random.rand(hosts) * 0.5 + 0.2  # Randomly generate load between 20-70%
print("random_host_loads\n", host_loads)
# Host load per attribute
#host_loads = np.random.rand(hosts, attrbutes) * 0.5

def schedule_vm(vm_attributes, host_attributes, attribute_weights, green_energy, host_loads, green_energy_weight=100, host_load_weight=100):
    # Adjust VM attributes based on weights
    weighted_vm_attributes = vm_attributes * attribute_weights
    
    # Calculate weighted host attributes
    weighted_host_attributes = host_attributes * attribute_weights
    print("weighted_host_attributes\n", weighted_host_attributes)
    
    # Calculate potential new load for each host after adding the VM, normalized by attribute importance
    potential_new_load = weighted_host_attributes + weighted_vm_attributes
    print("potential_new_load\n", potential_new_load)
    
    # Calculate host scores based on potential load, green energy, and current load
    # Lower scores are better. This is a simplified scoring mechanism; adjust weights as needed.
    scores = np.sum(potential_new_load, axis=1) - green_energy * green_energy_weight + host_loads * host_load_weight  # Weight green energy and load heavily
    print("scores\n", scores)
    
    # Choose the host with the lowest score
    chosen_host_idx = np.argmin(scores)
    
    return chosen_host_idx


def __main__():
    
    # While loop that waits for user input to schedule a VM
    while True:
        print("Press enter to schedule a VM")
        input()
        
        # Schedule the VM
        vm_attributes = generate_vm()
        print(f"VM attributes:\n {vm_attributes}")
        chosen_host_idx = schedule_vm(vm_attributes, host_attributes, attribute_weights, green_energy, host_loads, 0, 0)
        print(f"VM should be scheduled on host: {chosen_host_idx}")


host_attributes
 [[149.76270079 183.03787327 160.55267521 148.9766366  124.73095987]
 [169.17882261 127.51744225 218.35460016 232.7325521  116.68830377]
 [198.34500762 145.77898395 153.60891222 225.11932766  54.20721164]]
random_green_energy
 [0.0871293  0.0202184  0.83261985]
random_host_loads
 [0.58907838 0.63500607 0.68930917]


In [ ]:
import numpy as np

def interference_aware_scheduler(hosts, vm, attribute_weights, green_energy_weights, load_factor):
    """
    Schedules a VM onto a host using interference-aware scheduling

    Args:
        hosts (list): A list of dictionaries, each representing a host with the following keys:
            - 'attributes': A NumPy array representing the host's current attribute values (a1-aN).
            - 'current_load': A scalar representing the host's proportional load (0-1).
            - 'green_energy_metric': A scalar representing the host's green energy factor (higher is better).
        vm (dict): A dictionary representing the VM with the following keys:
            - 'attributes': A NumPy array representing the VM's attribute values (a1-aN).
        attribute_weights (np.ndarray): A weight vector for each attribute.
        green_energy_weights (float): Weight for the green energy metric.
        load_factor (float): Weight for the host's current load.

    Returns:
        int: The index of the selected host.
    """

    host_scores = []
    for host_index, host in enumerate(hosts):
        # Calculate potential new attribute values
        potential_attributes = host['attributes'] + vm['attributes']

        # Calculate distance from median (adjust how you calculate 'ideal_attributes' as needed)
        ideal_attributes = np.median(hosts, axis=0)['attributes']  
        attribute_distance = np.linalg.norm(potential_attributes - ideal_attributes)

        # Load penalty (higher load = worse)
        load_penalty = host['current_load'] * load_factor

        # Green energy bonus (higher green energy metric = better)
        green_energy_bonus = host['green_energy_metric'] * green_energy_weights

        # Weighted score calculation
        score = - (attribute_distance * attribute_weights + load_penalty) + green_energy_bonus
        host_scores.append(score)

    # Select the host with the highest score
    best_host_index = np.argmax(host_scores)
    return best_host_index

# --- EXAMPLE USAGE ---
hosts = [
    {'attributes': np.array([2, 5, 1]), 'current_load': 0.6, 'green_energy_metric': 0.3},
    {'attributes': np.array([4, 3, 4]), 'current_load': 0.2, 'green_energy_metric': 0.8},
    {'attributes': np.array([3, 1, 6]), 'current_load': 0.4, 'green_energy_metric': 0.5}
]
vm = {'attributes': np.array([1, 3, 2])}
attribute_weights = np.array([0.4, 0.2, 0.4])  # Example weights 
green_energy_weight = 0.5
load_factor = 1.0

selected_host_index = interference_aware_scheduler(hosts, vm, attribute_weights, green_energy_weight, load_factor)
print("Selected Host:", selected_host_index) 
